In [6]:
import numpy as np
import os,re,datetime
import pandas as pd
import pickle
from tools.MyClock import MyClock
import scipy.sparse
import warnings
from keras.models import load_model
warnings.filterwarnings('ignore')
clk = MyClock()

Using TensorFlow backend.


# save predictions

In [5]:
# copied from run_model.py
def get_input(in_dir):
    x_train = np.load(os.path.join(in_dir,'x_train.npy'))
    dirs = [os.path.join(in_dir,d) for d in sorted(os.listdir(in_dir)) if d.startswith('y_train')]
    y_trains = [scipy.sparse.load_npz(d).todense() for d in dirs]

    x_test = np.load(os.path.join(in_dir,'x_test.npy'))
    dirs = [os.path.join(in_dir,d) for d in sorted(os.listdir(in_dir)) if d.startswith('y_test')]
    y_tests = [scipy.sparse.load_npz(d).todense() for d in dirs]
    return x_train,y_trains,x_test,y_tests
def save_predictions(model,x_test,y_tests,out_dir):
    print('SAVE PREDICTIONS TO : {}'.format(out_dir))
    outputs = model.predict(x_test,verbose=1)
    ind_dirs = [os.path.join(out_dir,'pred_outputs{}.txt'.format(i)) for i in range(len(y_tests))]
    log_dirs = [os.path.join(out_dir,'pred_logits{}.txt'.format(i)) for i in range(len(y_tests))]
    f_ind = [open(ind_dir,'ab') for ind_dir in ind_dirs]
    f_log = [open(log_dir,'ab') for log_dir in log_dirs]
    for i,out_prob in enumerate(out_probs):
        ind = np.argsort(out_prob,axis=1)[:,:-11:-1]
        logits = np.take_along_axis(out_prob, ind, axis=1)
        np.savetxt(f_ind[i],ind,fmt='%d')
        np.savetxt(f_log[i],logits,fmt='%1.3f')
    for f in f_ind + f_log:
        f.close()

In [7]:
IN_DIR = 'data/dl_sic'
MODEL_DIR = 'outputs/190724_115500_xmlcnn'
_,_,x_test,y_tests = get_input(IN_DIR)
model = load_model(os.path.join(MODEL_DIR,'model.h5'))

W0724 12:10:13.706871 140317602686784 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 12:10:25.421075 140317602686784 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0724 12:10:25.460620 140317602686784 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0724 12:10:25.493466 140317602686784 deprecation_wrapper.py:119] From /home/angela/env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 12:10:25.494292 14031760

ValueError: Unknown loss function:binary_cross_entropy_with_logits

# get p@5 (classification)

In [14]:
IN_DIR = 'data/dl_sic'
MODEL_DIRS = [
    'outputs/190722_204831_xmlcnn',
#     'outputs/190715_215817_attentionxml',
#     'outputs/190715_235041_attention',
]

y_test = scipy.sparse.load_npz(os.path.join(IN_DIR,'y_test.npz'))
y_test = np.argmax(y_test.todense(),axis=1).A1
k=5
for MODEL_DIR in MODEL_DIRS:
    IND_DIR = os.path.join(MODEL_DIR,'prediction_10_ind.txt')
    pred = np.loadtxt(IND_DIR)[:,:k].astype(int)
    correct = 0
    for i,y in enumerate(y_test):
        if y in pred[i,:]:
            correct+=1
    print('DIR: {}, p@{}: {:.4f}'.format(MODEL_DIR.split('/')[-1],k,correct/len(y_test)))

DIR: 190722_204831_xmlcnn, p@5: 0.7466
